In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('data/Advertising.csv',index_col=0)

In [4]:
df.head()

,TV,radio,newspaper,sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [13]:
X = df[['TV', 'radio', 'newspaper']]
y = df['sales']
n = len(y)
X = np.append(np.ones((n,1)), X.values.reshape(n,3), axis = 1)
y = df['sales'].values.reshape(n,1)
par = np.zeros((4,1))

In [14]:
def cost_function(X, y , par):
    y_pred = np.dot(X, par)
    error = (y_pred - y)**2
    cost = 1/(n)*np.sum(error)
    return cost

In [15]:
def grad_d(X,y, par, alpha, iterations):
    costs = []
    for i in range(iterations):
        y_pred = np.dot(X, par)
        der = np.dot (X.transpose(), (y_pred - y))/ n
        par -= alpha * der
        costs.append(cost_function(X,y, par))
    return par, costs

In [16]:
par, costs = grad_d(X,y, par, 0.00005, 500000)

In [19]:
np.round(par, 3)

array([[ 2.863e+00],
       [ 4.600e-02],
       [ 1.890e-01],
       [-1.000e-03]])

In [157]:
import numpy as np
import statsmodels.api as sm
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.612
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                     312.1
Date:                Sat, 18 Jun 2022   Prob (F-statistic):           1.47e-42
Time:                        08:48:52   Log-Likelihood:                -519.05
No. Observations:                 200   AIC:                             1042.
Df Residuals:                     198   BIC:                             1049.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.0326      0.458     15.360      0.0

In [172]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X,y)
print(model.coef_)
print(model.intercept_)

[[0.         0.04753664]]
[7.03259355]


In [30]:
import seaborn as sns
df = sns.load_dataset('diamonds')

In [31]:
df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)
df

,carat,cut,color,clarity,price
0,0.23,Ideal,E,SI2,326
1,0.21,Premium,E,SI1,326
2,0.23,Good,E,VS1,327
3,0.29,Premium,I,VS2,334
4,0.31,Good,J,SI2,335
...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,2757
53936,0.72,Good,D,SI1,2757
53937,0.70,Very Good,D,SI1,2757
53938,0.86,Premium,H,SI2,2757


In [32]:
df = pd.get_dummies(df, drop_first=True)
df

,carat,price,cut_Premium,cut_Very Good,cut_Good,cut_Fair,color_E,color_F,color_G,color_H,color_I,color_J,clarity_VVS1,clarity_VVS2,clarity_VS1,clarity_VS2,clarity_SI1,clarity_SI2,clarity_I1
0,0.23,326,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
1,0.21,326,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2,0.23,327,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0
3,0.29,334,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
4,0.31,335,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,2757,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
53936,0.72,2757,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
53937,0.70,2757,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
53938,0.86,2757,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [33]:
df['carat'] = np.log(1+df['carat'])
df['price'] = np.log(1+df['price'])

In [34]:
X_cols = [col for col in df.columns if col!='price'] 
X = df[X_cols]
y = df['price']

In [38]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

params = {
    'learning_rate': ['constant'],
    'eta0': np.logspace(-4,-1,4),
    'max_iter': np.logspace(0,3,10),
    'loss': ['squared_loss', 'epsilon_insensitive'],
    'penalty': ['elasticnet'], 
    'alpha': np.logspace(-3,3,15),
    'l1_ratio': np.linspace(0,1,11)
}

In [43]:
sgd = SGDRegressor()

grid = GridSearchCV(estimator=sgd, param_grid=params, cv=5)
grid.fit(X_train, y_train)

c:\Users\Asya\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:165: FutureWarning: The loss 'squared_loss' was deprecated in v1.0 and will be removed in version 1.2. Use `loss='squared_error'` which is equivalent.
  warnings.warn(
c:\Users\Asya\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
c:\Users\Asya\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:165: FutureWarning: The loss 'squared_loss' was deprecated in v1.0 and will be removed in version 1.2. Use `loss='squared_error'` which is equivalent.
  warnings.warn(
c:\Users\Asya\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
c:\Users\Asya\anacon

KeyboardInterrupt: 

In [90]:
def func_val(x):
    r = 24*x**2-4*x
    return r

def diff_val(x):
    r = 48*x-4
    return r

In [54]:
diff_val(x0)

-1.3369999999999997

In [92]:
x0 = 42
eps = 0.0001
list_x=[x0]

while (abs(func_val(x0))>eps or x0<=0):
    x1 = x0-func_val(x0)/diff_val(x0)
    list_x.append(x1)
    x0=x1
    
print(f'Значение функции в точке {x0:.3f} равно {func_val(x0)}')
    


list_x   

Значение функции в точке 0.167 равно 5.6345204502150636e-08


[42,
 21.041749502982107,
 10.562707090133793,
 5.323351550447383,
 2.7040050774153417,
 1.3949941413301903,
 0.7418109325525483,
 0.41784523900811205,
 0.26096925221473555,
 0.19169814030401197,
 0.16955770984744145,
 0.1667151339969682,
 0.1666666807529666]

In [74]:
def newtons_method(f, der, eps, init):
    iter_count = 0
    x_curr = init
    f = func_val(x_curr)
    f_prime = diff_val(x_curr)
    while (abs(f) > eps):
        f = f(x_curr)
        f_der = der(x_curr)
        x_curr = x_curr - (f)/(f_prime)
        iter_count += 1
    return x_curr
 
from scipy.optimize import newton
newton(func=func_val,x0=12,fprime=diff_val, tol=0.0001)

9.727134419408875

In [82]:
x0

9.727134419408875

In [86]:
func_val(10.211)

109.45302393100019